# Dataset Compilation
This notebook walks you through how to compile and clean a dataset. We will use the USGS dataRetrieval tool to compile flow and water quality data. We will then compile watershed characteristics.

The generalized approach I like to take to data cleaning are:
1. Remove irrelevant data
2. Remove structural errors
3. Aggregate or remove duplicates
4. Fill or remove missing values
5. Remove outliers


## Set up
I use Anaconda as a package manager because it simplifies package management, dependencies, and deployment for you. I also like the built in applications, including Spyder (my preferred IDE because it's UI is most similar to R Studio and MATLAB.) and Jupyter Notebooks. 
  
See README markdown for details. 

In [4]:
# Import all libraries at the top of your code so you can easily see and organize all the packges you are using. 
import pandas as pd
import numpy as np
from dataretrieval import nwis, wqp
import os
from pathlib import Path

**TIP: File and folder organization**

My preferred approach to folder organization is to have 3 folders: 
- INPUT: All my raw input data. I will rarely save edited files here.
- OUTPUT: All my code outputs and intermediate files.
- CODE: All my code will be in this folder. This is the folder I push to git. I like to label the use of my codes based on their function. Example of this
    - 'DATA' for scripts that organize data.
    - "ANA" for scripts that are used for analysis.
    - "FIG" for scripts that generate figures.
    - "MODEL" for model wrapper scripts.
    - "FUN" for functions that other scripts call.

When calling files from INPUTS or OUTPUT folders, using absolute paths is more reliable and easier to debug. Relative paths are more flexible and often requires smaller blocks of code. However, it's good practice to use absolute paths but ultimately it's based on preference. 

In [6]:
inputDataFilepath ='C:/Users/danyk/Work/4_Data_Science/DataExplorationWorkshop/INPUTS/'
outputDataFilepath ='C:/Users/danyk/Work/4_Data_Science/DataExplorationWorkshop/OUTPUT/'

## Initializing dataset

We will start by initializing the data setting up the list of watersheds using the watershed attribute data. Since we will want all our sites to have watershed characteristics, we will use that dataset to subset all available flow and water quality sites.

In [7]:
BasinData = pd.read_csv(inputDataFilepath+'Dataset1_BasinID/BasinID.txt', sep=",", dtype={'STAID': str})

# Always useful to check your data. In this case, it's important to note that 'STAID' need to be read as strings because of their leading '0's. 
BasinData.head()

,STAID,STANAME,DRAIN_SQKM,HUC02,LAT_GAGE,LNG_GAGE,STATE,HCDN-2009,CLASS,AGGECOREGION
0,01011000,"Allagash River near Allagash, Maine",3186.8,01,47.069611,-69.079544,ME,NaN,Non-ref,NorthEast
1,01013500,"Fish River near Fort Kent, Maine",2252.7,01,47.237394,-68.582642,ME,yes,Ref,NorthEast
2,01015800,"Aroostook River near Masardis, Maine",2313.8,01,46.523003,-68.371764,ME,NaN,Non-ref,NorthEast
3,01016500,"MACHIAS RIVER NEAR ASHLAND, ME",847.8,01,46.628311,-68.434792,ME,NaN,Non-ref,NorthEast
4,01017000,"Aroostook River at Washburn, Maine",4278.9,01,46.777294,-68.157194,ME,NaN,Non-ref,NorthEast


It's good practice to check the columns when you read in a new dataframe.
- DRAIN_SQKM: Watershed area in square km.
- LAT and LONG of the gage/pour point.
- HUC02: What HUC region does this watershed in.
- CLASS: GAGES reference or non-ref watershed. Ref watersheds are watersheds with a lower human impact (agriculture, hydrology changes, etc.)
- AGGECOREGION: Ecoregion based on topology.

I am going to rename them to my prefered headers. But feel free to leave them. 

In [60]:
BasinData = BasinData.rename(columns={"STAID": "SITE", 
                                          "STANAME": "NAME", 
                                          "DRAIN_SQKM": "AREA_SQKM",
                                          "LAT_GAGE": "LAT", 
                                          "LNG_GAGE": "LONG"})
BasinData.head()

,SITE,NAME,AREA_SQKM,HUC02,LAT,LONG,STATE,HCDN-2009,CLASS,AGGECOREGION
0,01011000,"Allagash River near Allagash, Maine",3186.8,01,47.069611,-69.079544,ME,NaN,Non-ref,NorthEast
1,01013500,"Fish River near Fort Kent, Maine",2252.7,01,47.237394,-68.582642,ME,yes,Ref,NorthEast
2,01015800,"Aroostook River near Masardis, Maine",2313.8,01,46.523003,-68.371764,ME,NaN,Non-ref,NorthEast
3,01016500,"MACHIAS RIVER NEAR ASHLAND, ME",847.8,01,46.628311,-68.434792,ME,NaN,Non-ref,NorthEast
4,01017000,"Aroostook River at Washburn, Maine",4278.9,01,46.777294,-68.157194,ME,NaN,Non-ref,NorthEast


## Compiling water quality and discharge data

The USGS has a library called 'dataRetrieval' which helps with loading hydrologic and water quality data into Python. dataRetrieval library was originally built for R, and has better documentation. But they do have a python version, which is what we will use. 

We want to compile inorganic nitrogen fraction and discharge. With this we see that we have dissolved fraction of Inorganic nitrogen (nitrate and nitrite) and discharge. The units are likely mg/L as N and ft3/s, but we need to double check. You always want to double check your data to make sure you are pulling the **right** data and that you understand **what** you are pulling. It sounds trivial, but this is how mistakes happen. 

In [197]:
# What solutes do we want to pull? USGS has parameter codes for discharge, solute, and each of their solutes
N_paramCd = 'Inorganic nitrogen (nitrate and nitrite)'
Q_paramCd = '00060'

In [198]:
# Over what period? We are defining this to only pull data within this timeframe. 
# We are defining different data structures because the functions have different data structures. 
N_startDate = "01-01-1980"
N_endDate = "12-31-2020"
Q_startDate = "1980-01-01"
Q_endDate = "2020-12-31"

### Dissolved Inorganic Nitrogen Concentration Data
Now that we have the stations we will be using, we wnat to find which ones have nitrate concentration data and discharge data. We will start by pulling inorganic nitrogen concentration data because there are fewer stations with available data. 

This part takes a while to run.

In [199]:
# Now we want to read in the data. We will read in nitrate solute data first, because these data are a lot more sparse than discharge. 

# There are too many sites to call all at once, so we will chunk the list of sites. 
siteNumbers = MetricTable['SITE'].tolist()
split_siteNumbers = np.array_split(siteNumbers, 50)
rawDailyWQData_l = []

# loop through chunks and make requests 
for site_list_a in split_siteNumbers:
    #site_list = site_list_a.tolist()
    site_list = ['USGS-' + site for site in site_list_a]
    data, metadata = wqp.get_results(siteid=site_list, startDateLo=N_startDate, startDateHi=N_endDate,characteristicName=N_paramCd)
    rawDailyWQData_l.append(data)  # Append results to list

rawDailyWQData = pd.concat(rawDailyWQData_l, ignore_index=True)

C:\Users\danyk\anaconda3\envs\dataExploration\lib\site-packages\dataretrieval\wqp.py:494: UserWarning: USGS discrete water quality data availability and format are changing. Beginning in March 2024 the data obtained from legacy profiles will not include new USGS data or recent updates to existing data. To view the status of changes in data availability and code functionality, visit: https://doi-usgs.github.io/dataRetrieval/articles/Status.html. If you have additional questions about these changes, email CompTools@usgs.gov.
  warnings.warn(('USGS discrete water quality data availability '
C:\Users\danyk\anaconda3\envs\dataExploration\lib\site-packages\dataretrieval\wqp.py:494: UserWarning: USGS discrete water quality data availability and format are changing. Beginning in March 2024 the data obtained from legacy profiles will not include new USGS data or recent updates to existing data. To view the status of changes in data availability and code functionality, visit: https://doi-usgs.gi

In [200]:
pd.set_option('display.max_columns', None)
rawDailyWQData.head(8)

,OrganizationIdentifier,OrganizationFormalName,ActivityIdentifier,ActivityTypeCode,ActivityMediaName,ActivityMediaSubdivisionName,ActivityStartDate,ActivityStartTime/Time,ActivityStartTime/TimeZoneCode,ActivityEndDate,ActivityEndTime/Time,ActivityEndTime/TimeZoneCode,ActivityDepthHeightMeasure/MeasureValue,ActivityDepthHeightMeasure/MeasureUnitCode,ActivityDepthAltitudeReferencePointText,ActivityTopDepthHeightMeasure/MeasureValue,ActivityTopDepthHeightMeasure/MeasureUnitCode,ActivityBottomDepthHeightMeasure/MeasureValue,ActivityBottomDepthHeightMeasure/MeasureUnitCode,ProjectIdentifier,ActivityConductingOrganizationText,MonitoringLocationIdentifier,ActivityCommentText,SampleAquifer,HydrologicCondition,HydrologicEvent,SampleCollectionMethod/MethodIdentifier,SampleCollectionMethod/MethodIdentifierContext,SampleCollectionMethod/MethodName,SampleCollectionEquipmentName,ResultDetectionConditionText,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/MeasureUnitCode,MeasureQualifierCode,ResultStatusIdentifier,StatisticalBaseCode,ResultValueTypeName,ResultWeightBasisText,ResultTimeBasisText,ResultTemperatureBasisText,ResultParticleSizeBasisText,PrecisionValue,ResultCommentText,USGSPCode,ResultDepthHeightMeasure/MeasureValue,ResultDepthHeightMeasure/MeasureUnitCode,ResultDepthAltitudeReferencePointText,SubjectTaxonomicName,SampleTissueAnatomyName,ResultAnalyticalMethod/MethodIdentifier,ResultAnalyticalMethod/MethodIdentifierContext,ResultAnalyticalMethod/MethodName,MethodDescriptionText,LaboratoryName,AnalysisStartDate,ResultLaboratoryCommentText,DetectionQuantitationLimitTypeName,DetectionQuantitationLimitMeasure/MeasureValue,DetectionQuantitationLimitMeasure/MeasureUnitCode,PreparationStartDate,ProviderName
0,USGS-ME,USGS Maine Water Science Center,nwisma.01.98100751,Sample-Routine,Water,Surface Water,1980-10-20,15:00:00,EDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Geological Survey-Water Resources Discipline,USGS-01049265,NaN,NaN,Not determined,Routine sample,USGS,USGS,USGS,Unknown,NaN,Inorganic nitrogen (nitrate and nitrite),Total,0.12,mg/l as N,NaN,Historical,NaN,Actual,NaN,NaN,NaN,NaN,NaN,NaN,630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
1,USGS-ME,USGS Maine Water Science Center,nwisma.01.98100751,Sample-Routine,Water,Surface Water,1980-10-20,15:00:00,EDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Geological Survey-Water Resources Discipline,USGS-01049265,NaN,NaN,Not determined,Routine sample,USGS,USGS,USGS,Unknown,NaN,Inorganic nitrogen (nitrate and nitrite),Dissolved,0.13,mg/l as N,NaN,Historical,NaN,Actual,NaN,NaN,NaN,NaN,NaN,NaN,631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
2,USGS-MA,USGS Massachusetts Water Science Center,nwisma.01.98001110,Sample-Routine,Water,Surface Water,1980-04-15,15:15:00,EST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Geological Survey-Water Resources Discipline,USGS-01103500,NaN,NaN,Not determined,Routine sample,USGS,USGS,USGS,Unknown,NaN,Inorganic nitrogen (nitrate and nitrite),Total,0.14,mg/l as N,NaN,Historical,NaN,Actual,NaN,NaN,NaN,NaN,NaN,NaN,630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
3,USGS-MA,USGS Massachusetts Water Science Center,nwisma.01.98001110,Sample-Routine,Water,Surface Water,1980-04-15,15:15:00,EST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Geological Survey-Water Resources Discipline,USGS-01103500,NaN,NaN,Not determined,Routine sample,USGS,USGS,USGS,Unknown,NaN,Inorganic nitrogen (nitrate and nitrite),Dissolved,0.15,mg/l as N,NaN,Historical,NaN,Actual,NaN,NaN,NaN,NaN,NaN,NaN,631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NWIS
4,USGS-MA,USGS Massachusetts Water Science Center,nwisma.01.98001114,Sample-Routine,Water,Surface Water,1980-06-24,12:30:00,EDT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Geological Survey-Water Resources Discipline,USGS-01103500,NaN,NaN,Not determined,Routine sample,USGS,USGS,USGS,Unknown,NaN,Inorgan

Take a look at the columns from the water quality data. There is a lot of information included and I suggest you go through these columns before you use the data! It's crucial that you check your data so you can check if any columns has relevant information.
Thing to note:
- 'ActivityStartDate' has 2 entries every day. Strange! 
- 'ResultSampleFraction' we can see that there isas both 'dissolved' and 'total' (unfiltered)... These are two different solutes. We need to investigate this.

But for now, since we are assuming a familiarity with the data, we'll just isolate the columns we want and move on. 
- 'ActivityStartDate': Date of sample collected
- 'MonitoringLocationIdentifier': Site ID
- 'ResultSampleFraction': Type of DIN 
- 'ResultMeasureValue': Measured value
- 'ResultMeasure/MeasureUnitCode': Units

#### Data cleaning
Data cleaning is often a tedious and time consuming step, but arguably one of the most important one. Spending time on the front end cleaning your data will save you time and energy down the line.  

##### Getting to know your data
Starting off, you might want to just check the unique values (categorical) or the max and mix (numeric) of the values in each relevant columns.

In [387]:
# Data is messy! Check it and clean it. This is our first (and definitely not our last) use of  Split-Apply-Combine.
uncleanDailyWQData = rawDailyWQData[['MonitoringLocationIdentifier', 'ActivityStartDate','ResultMeasureValue','ResultSampleFractionText','ResultMeasure/MeasureUnitCode']]
uncleanDailyWQData = uncleanDailyWQData.rename(columns={"MonitoringLocationIdentifier": "USGSSite", 
                                                        "ActivityStartDate": "Date", 
                                                        "ResultMeasureValue": "Conc", 
                                                        "ResultSampleFractionText": "Fraction", 
                                                        "ResultMeasure/MeasureUnitCode": "Units"})

DailyWQData = uncleanDailyWQData.copy()

# Converting the date to datetime type
DailyWQData['Date'] = pd.to_datetime(DailyWQData['Date'],
               format='%Y-%m-%d')

# There are a few values we can remove off the hop. We don't want NA values, zeroes, or negatives. 
DailyWQData = DailyWQData.dropna(subset=['Conc'])
DailyWQData = DailyWQData.loc[DailyWQData['Conc'] > 0]
DailyWQData.head(10)

,USGSSite,Date,Conc,Fraction,Units
0,USGS-01049265,1980-10-20,0.12,Total,mg/l as N
1,USGS-01049265,1980-10-20,0.13,Dissolved,mg/l as N
2,USGS-01103500,1980-04-15,0.14,Total,mg/l as N
3,USGS-01103500,1980-04-15,0.15,Dissolved,mg/l as N
4,USGS-01103500,1980-06-24,0.30,Total,mg/l as N
5,USGS-01103500,1980-06-24,0.30,Dissolved,mg/l as N
6,USGS-01022500,1980-01-22,0.05,Total,mg/l as N
7,USGS-01022500,1980-01-22,0.06,Dissolved,mg/l as N
8,USGS-01103500,1980-09-09,0.06,Total,mg/l as N
9,USGS-01103500,1980-09-09,0.05,Dissolved,mg/l as N


In [388]:
grouped_solute = DailyWQData.groupby('Fraction').agg({
    'Date': ['min', 'max'],
    'Conc': ['min', 'mean', 'max']
})
grouped_solute.head()

Date              Conc                  
                    min        max    min       mean    max
Fraction                                                   
Bed Sediment 1980-01-02 2014-11-24  0.920  11.348767  240.0
Dissolved    1980-01-02 2020-12-31  0.001   1.527721  637.0
Total        1980-01-02 2020-12-28  0.001   1.577619  250.0

In [389]:
grouped_solute = DailyWQData.groupby('USGSSite').agg({
    'Conc': ['min', 'mean', 'max']
})
grouped_solute = grouped_solute.sort_values(by=('Conc', 'max'), ascending=False)
grouped_solute.head(10)

Conc                  
                 min       mean    max
USGSSite                              
USGS-04199000  0.140   4.400685  637.0
USGS-07019185  0.030   4.270769  250.0
USGS-07249400  0.010   1.285598  240.0
USGS-02336526  0.044   1.609385  230.0
USGS-08010000  0.050   3.637500  130.0
USGS-09431500  0.020   1.084680  130.0
USGS-07189540  0.467  11.270645  117.0
USGS-04212100  0.044   0.548967   97.1
USGS-06893300  0.630   6.791250   86.0
USGS-04193500  0.020   5.423071   65.4

In [390]:
grouped_solute = grouped_solute.sort_values(by=('Conc', 'min'), ascending=True)
grouped_solute.head(10)

Conc                 
                 min      mean    max
USGSSite                             
USGS-12395000  0.001  0.036493  0.280
USGS-10336698  0.001  0.017617  0.439
USGS-02256500  0.001  0.053062  0.300
USGS-13345000  0.001  0.367242  2.500
USGS-02208150  0.001  0.439319  4.500
USGS-14138900  0.001  0.057300  0.130
USGS-10336674  0.001  0.021365  0.240
USGS-10336660  0.001  0.026308  0.304
USGS-10336676  0.001  0.020598  0.550
USGS-09011000  0.001  0.053441  0.317

Three takeaways:
- Station identifiers have 'USGS' in front of them.
- We definitely do not want bed sediment. Remove that!
- We want dissolved and the number of datapoints are similar, so we can just take dissolve.
- There are some funky values... We should probably deal  with the outliers.

In [391]:
# We are using Boolean Indexing to remove all other fractions. You can also use the 'drop' function. 
DailyWQData = DailyWQData[DailyWQData['Fraction'] == 'Dissolved']

In [392]:
# Let's check the units to make sure they are what we expect and that they are consistent through the entire dataset. 
unitsAvail = DailyWQData['Units'].unique()
print(unitsAvail)

['mg/l as N']


The only units available are 'mg-N / L' which is what we want! Great, we don't need to convert anything. 

##### Removing Outliers

**Removing outliers is more an art than a science.**

Removing data from a dataset can be tricky. In part because you do not want to bias your data by removing concentrations that hold information about the watershed. For example, for a nitrogen-rich watershed low flow (baseflow) might have really high concentrations. If we were to remove those concentrations, we could lose all the information about the low flow part of the watershed's hydrograph! 

Those maximum values are also very high for mg/L as N! background concentrations in rivers is about 1 mg-N/L, but it can be lower. Maximum concentrations are around 30 mg-N/L. Therefore, concentrations a whole magnitude above is definitely problematic. We need to remove those. 

We also see minimum concentrations at 0.001 mg-N/L. That is likely the lower detection limit for the instruments. While there are standards set by EPA these limits change based on instrumentation used. Based on what I could find, currently for the USGS, the limit is 0.02 mg-N/L. We don't want to remove these data because they still hold information, but it can mess up your statistics. So let's quantify how non-detects each station has, in case we want to remove some stations from our dataset. 

There are many different ways you can remove outliers, and some are better suited for some data distributions than others. We know that a lot of envirionmental/hydrological data has a log-distributions (skewed and heavy tails), thus removing outliers based on a normal distribution might remove non-outliers. Try out a few different approaches and pick the one that maintains the distribution of your data but removes some values that are likely erroneous. 

In [393]:
# Removing high values
uniqSites = DailyWQData['USGSSite'].unique()
rmIdx= []
for uniqSite in uniqSites:
    temp = DailyWQData[DailyWQData['USGSSite'] == uniqSite].copy()
    
    # Calculating log10 concentration. 
    temp['LogConc'] = np.log10(temp['Conc'])

    # Finding the thresholds of concentration for this specific site. 
    mean = temp['LogConc'].mean()
    std_dev = temp['LogConc'].std()
    lowerThreshold = mean - 2 * std_dev
    upperThreshold = mean + 2 * std_dev
    
    # Indices are from DailyWQData. So we can collect indices and filter data after the loop.
    rmIdx_t = temp[(temp['LogConc'] >= lowerThreshold) & (temp['LogConc'] <= upperThreshold)].index
    rmIdx.extend(rmIdx_t)
    
DailyWQData = DailyWQData.loc[rmIdx]

In [394]:
grouped_solute = DailyWQData.groupby('USGSSite').agg({
    'Conc': ['min', 'mean', 'max']
})
grouped_solute = grouped_solute.sort_values(by=('Conc', 'min'), ascending=True)
grouped_solute.head(10)

Conc                 
                  min      mean    max
USGSSite                              
USGS-14138900   0.001  0.042167  0.120
USGS-10296500   0.002  0.042833  0.149
USGS-10308200   0.002  0.009000  0.030
USGS-10336698   0.002  0.015246  0.073
USGS-10336730   0.002  0.016292  0.095
USGS-103367592  0.002  0.032610  0.257
USGS-10336676   0.002  0.015161  0.096
USGS-10336770   0.002  0.006283  0.021
USGS-10336674   0.002  0.018480  0.091
USGS-10336775   0.002  0.009294  0.034

In [395]:
# Assessing detection limits

def pct_thresholdLimit(sitesTS, threshold):
    return (sitesTS <= threshold).sum()/len(sitesTS)

threshold =  0.002
grouped_solute = DailyWQData.groupby('USGSSite')['Conc'].agg(Pct_LowDetLim=lambda x: pct_thresholdLimit(x, threshold))
grouped_solute = grouped_solute.sort_values(by=('Pct_LowDetLim'), ascending=False)
grouped_solute.head(20)

,Pct_LowDetLim
USGSSite,
USGS-14138900,0.333333
USGS-10308200,0.222222
USGS-10337500,0.216216
USGS-103366097,0.196429
USGS-10336626,0.142857
USGS-10336770,0.127389
USGS-10336780,0.125000
USGS-10336676,0.113158
USGS-05129115,0.111111


There are 10 stations that have concentrations below the detection limit for over 10% of their timeseries. I personally don't want to remove them now, because I might still be able to make sure of those data. But it's important to keep this value in mind moving forward!

In [396]:
print(len(DailyWQData)/len(uncleanDailyWQData))

0.5739667395561063


After cleaning, we have just nearly 60% of our data remaining. Nice! 

Now we also have to remove the USGS sigifier in front of the site ID for consistency with our basin data and because USGS data from NWIS data does not have that identifier.

In [397]:
DailyWQData['Site'] = DailyWQData['USGSSite'].str.slice(start=5)
col = DailyWQData.pop("Site")
DailyWQData.insert(0, col.name, col)
DailyWQData.head(1)

,Site,USGSSite,Date,Conc,Fraction,Units
1,01049265,USGS-01049265,1980-10-20,0.13,Dissolved,mg/l as N


Great! Moving along...

### Discharge
Now that we have the list of our final set of water quality stations, let's compile the flow data.
USGS records daily flow data, but only water quality sampled monthly, at most. We will use the water quality dataframe to pull only the relevant data.

We will be using the parameter code set at the start of the script "00060" with is the USGS parameter for daily mean discharge. Let's check the units!

In [208]:
qInfo, md = nwis.get_pmcodes(parameterCd='00060', partial=False)
qInfo.head()

,parameter_cd,group,parm_nm,epa_equivalence,result_statistical_basis,result_time_basis,result_weight_basis,result_particle_size_basis,result_sample_fraction,result_temperature_basis,CASRN,SRSName,parm_unit
0,00060,Physical,"Discharge, cubic feet per second",Not checked,Mean,1 Day,NaN,NaN,NaN,NaN,,"Stream flow, mean. daily",ft3/s


In [209]:
# Now let's pull the data and organize the dataframe how we want it. 
uniqSites = DailyWQData['Site'].unique()

rawDailyQData_l = []
for uniqSite in uniqSites:

    # Isolate the date
    WQ_dates = DailyWQData.loc[DailyWQData['Site'] == uniqSite, 'Date']
    
    # Pull Q data from USGS
    rawQ, md = nwis.get_dv(sites=uniqSite, parameterCd=Q_paramCd, start=Q_startDate, end=Q_endDate)
    rawQ.reset_index(inplace=True)
    rawQ.rename(columns={'index': 'datetime'}, inplace=True)
    rawQ['Date'] = rawQ['datetime'].dt.date
    rawQ['Date'] = pd.to_datetime(rawDailyQ['Date'],
                                  format='%Y-%m-%d')
    rawQ = rawQ[['site_no', 'Date', '00060_Mean']]
    
    # Only keep the data that has the same dates as WQ samples.
    WQ_dates = DailyWQData.loc[DailyWQData['Site'] == uniqSite, 'Date']
    rawQ = rawQ[rawQ['Date'].isin(WQ_dates)]
    
    rawDailyQData_l.append(rawQ)

rawDailyQData_l = pd.concat(rawDailyQData_l, ignore_index=True)
rawDailyQData_l.head()

In [375]:
# Organizing the dataframe to only include the data I want. 
uncleanDailyQData = rawDailyQData[['site_no', 'Date', '00060_Mean']]
uncleanDailyQData = uncleanDailyQData.rename(columns={"site_no": "Site", 
                                                        "00060_Mean": "Q_ft3s"})
DailyQData = uncleanDailyQData.copy()


def process_site_data(group):
    uniqSite = group['Site'].iloc[0]  # Get the unique site identifier from the group
    WQ_dates = group['Date']  # Isolate the dates
    
    # Pull Q data
    rawQ, md = nwis.get_dv(sites=uniqSite, parameterCd=Q_paramCd, start=Q_startDate, end=Q_endDate)
    rawQ.reset_index(inplace=True)
    rawQ.rename(columns={'index': 'datetime'}, inplace=True)
    rawQ['Date'] = pd.to_datetime(rawQ['datetime']).dt.date  # Convert to datetime and extract date

    # Filter to only keep data that matches WQ sample dates
    rawQ = rawQ[rawQ['Date'].isin(WQ_dates)]

    return rawQ

processed_data = DailyWQData.groupby('Site').apply(process_site_data)


#### Data cleaning
##### Getting to know your data
We want to deploy a similar data exploration and cleaning techniques from the concentration to the discharge, although it's a bit more straightfoward. 
Also, recall we want to convert from ft3/s to m3/s. 

In [376]:
# Removing zeroes and nans. In some circumstances you might want zero flows. 
# But for water quality analysis, how do you interpret a concentration with zero flow?
DailyQData = DailyQData.dropna(subset=['Q_ft3s'])
DailyQData = DailyQData.loc[DailyQData['Q_ft3s'] > 0]

In [377]:
grouped_Q = DailyQData.groupby('Site').agg({
    'Date': ['min', 'max'],
    'Q_ft3s': ['min', 'mean', 'max']
})
grouped_Q = grouped_Q.sort_values(by=('Q_ft3s', 'max'), ascending=False)
grouped_Q.head()

Date             Q_ft3s                        
                min        max     min          mean       max
Site                                                          
14211720 1980-01-16 2020-12-15  3150.0  34395.513834  240000.0
02035000 1980-01-15 2020-12-07   678.0  15814.109589  199000.0
03085000 1980-01-09 2000-09-29  1430.0  14813.414634  188000.0
01540500 1980-01-08 2009-08-18  1310.0  23720.921053  170000.0
05465500 1980-01-22 2020-12-17   680.0  14596.879433  159000.0

Okay, those are some pretty high discharges... but using absolute terms is challenging to interpret. Normalizing these discharges using watershed area can give us a better intuitive sense of the data and whether these magnitudes are possible.

Recall we wanted to convert our discharge into into m3/s? Let's do that now. Then we will use the Drainage area from the Metric Table to find the runoff (mm) for these flows. Figuring out what is an outlier is, again, kind of tricky because the threshold is up to you. 

In [378]:
len(DailyQData)

135299

In [382]:
# Convert to m3/s 
DailyQData['Q_m3s'] = DailyQData['Q_ft3s']*(0.3048**3)
DailyQData.head()

# Normalize using watershed area
# Site, Name, Area_SQKM
FlowNorm = pd.merge(DailyQData, BasinData, on='Site', how='inner')
FlowNorm['Q_mmd'] = (FlowNorm['Q_m3s'] / FlowNorm['Area_SQKM'])*(1/1000000)*1000*86400 # m2/km2 * mm/m * sec/day
FlowNorm.sort_values(by='Q_mmd',ascending=False, inplace=True)
FlowNorm.head(10)

,Site,Date,Q_ft3s,Q_m3s,Name,Area_SQKM,HUC02,Lat,Long,STATE,HCDN-2009,CLASS,AGGECOREGION,Q_mmd
99199,07362587,1998-12-21,6620.0,187.457524,"Alum Fork Saline River near Reform, AR",69.8,08,34.797591,-92.933507,AR,yes,Ref,EastHghlnds,232.039113
116960,10336660,1997-01-01,2000.0,56.633693,BLACKWOOD C NR TAHOE CITY CA,29.8,16,39.107407,-120.162135,CA,yes,Ref,WestMnts,164.199701
116961,10336660,1997-01-02,1970.0,55.784188,BLACKWOOD C NR TAHOE CITY CA,29.8,16,39.107407,-120.162135,CA,yes,Ref,WestMnts,161.736706
121833,10336676,1997-01-01,1390.0,39.360417,WARD C AT HWY 89 NR TAHOE PINES CA,24.7,16,39.132129,-120.157691,CA,yes,Ref,WestMnts,137.681782
121834,10336676,1997-01-02,1370.0,38.794080,WARD C AT HWY 89 NR TAHOE PINES CA,24.7,16,39.132129,-120.157691,CA,yes,Ref,WestMnts,135.700749
117208,10336660,2005-12-31,1340.0,37.944574,BLACKWOOD C NR TAHOE CITY CA,29.8,16,39.107407,-120.162135,CA,yes,Ref,WestMnts,110.013800
87604,07053250,2004-01-14,5680.0,160.839689,"Yocum Creek near Oak Grove, AR",136.1,11,36.454440,-93.356110,AR,NaN,Ref,EastHghlnds,102.105431
95828,07197000,2011-04-25,32900.0,931.624253,"Baron Fork at Eldon, OK",808.4,11,35.921200,-94.838563,OK,yes,Ref,EastHghlnds,99.569935
122079,10336676,2005-12-31,947.0,26.816054,WARD C AT HWY 89 NR TAHOE PINES CA,24.7,16,39.132129,-120.157691,CA,yes,Ref,WestMnts,93.801905
127742,12040500,1983-11-16,44100.0,1248.772935,"QUEETS RIVER NEAR CLEARWATER, WA",1153.4,17,47.537859,-124.315746,WA,yes,Ref,WestMnts,93.544288


In [381]:
FlowNorm.sort_values(by='Q_mmd',ascending=True, inplace=True)
FlowNorm.head(10)

,Site,Date,Q_ft3s,Q_m3s,Name,Area_SQKM,HUC02,Lat,Long,STATE,HCDN-2009,CLASS,AGGECOREGION,Q_mmd
94496,07241000,2004-08-17,0.03,0.000850,"North Canadian River blw Lk Overholser nr OKC, OK",34927.9,11,35.478669,-97.663375,OK,NaN,Non-ref,WestPlains,0.000002
93396,07234000,1981-03-31,0.03,0.000850,"Beaver River at Beaver, OK",20720.8,11,36.822248,-100.519316,OK,NaN,Non-ref,WestPlains,0.000004
78579,06473700,1985-03-29,0.01,0.000283,"Snake Creek near Ashton, SD",6828.4,10U,44.959142,-98.515650,SD,NaN,Ref,WestPlains,0.000004
103454,08123850,1996-07-23,0.07,0.001982,"Colorado Rv abv Silver, TX",38861.5,12,32.053744,-100.762052,TX,NaN,Non-ref,WestPlains,0.000004
58983,05053000,1981-09-03,0.01,0.000283,"WILD RICE RIVER NR ABERCROMBIE, ND",5414.6,09,46.468017,-96.783691,ND,NaN,Non-ref,CntlPlains,0.000005
76862,06452000,1991-01-07,0.05,0.001416,WHITE R NEAR OACOMA SD,25791.0,10U,43.748327,-99.556495,SD,yes,Ref,WestPlains,0.000005
99779,07311900,2002-12-10,0.01,0.000283,"Wichita Rv nr Seymour, TX",4771.7,11,33.700370,-99.388691,TX,NaN,Ref,WestPlains,0.000005
74733,06365900,1987-05-18,0.01,0.000283,"CHEYENNE RIVER NR DULL CENTER, WY",3981.1,10U,43.429137,-105.045807,WY,NaN,Non-ref,WestPlains,0.000006
104202,08099500,1994-01-11,0.01,0.000283,"Leon Rv nr Hasse, TX",3329.2,12,31.957924,-98.459212,TX,NaN,Non-ref,WestPlains,0.000007
74588,06324970,1992-09-21,0.01,0.000283,"LITTLE POWDER RIVER AB DRY CREEK, NEAR WESTON, WY",3204.0,10U,44.926929,-105.353327,WY,NaN,Non-ref,WestPlains,0.000008


We see high flows going up to 200 mm/d. That's a lot, but it's not impossible if there was storm. Take the 2nd entry 'BLACKWOOD C NR TAHOE CITY CA', in 1997 there was a rain storm in January after snowpack in December was more than 180 percent it's normal magnitude. So, my gut says these values are OK.

The low values also look ok especially when we look at the states these low flows occur in -- they are all arid regions! 

### Final words on concentration and flow data
There might be other issues with the data. QAQC process never really ends, and you might find issues with the data down the line which makes you have to rerun your scripts. It's always important to keep a skeptical eye toward your data. But these first passes allow you to clean your dataset to a reasonable standard so you aren't working with junk.

We are ready to compile the our three tables to generate our final dataset. 

In [70]:
CQ_Data = pd.merge(BasinData, DailyQData, on='Site', how='outer')
CQ_Data = pd.merge(CQ_Data, DailyWQData, on=['Site','Date'], how='inner')
CQ_Data = CQ_Data[['Site', 'Name', 'Area_SQKM', 'HUC02','Lat','Long','STATE','HCDN-2009','CLASS','AGGECOREGION', 'Date', 'Q_m3s', 'Conc']]

# Let's make sure they merged correctly by checking for NaNs in flow and concentration. 
print(CQ_Data[['Q_m3s','Conc']].isnull().any().any())

CQ_Data.to_csv(inputDataFilepath+'CQ_Data_20240602.txt',sep=',', index=False)

NameError: name 'DailyQData' is not defined

## Compiling watershed attributes
Let's not compile the watershed attributes. The same data cleaning principles apply, however, since these are all published datasets, we can anticipate there to be fewer errors in the raw data. However, errors can be introduced at any point so you should always QAQC your data.

These data are in 'INPUT/' folder. Datasets labeled as 'Dataset99_XX' are derived from non-USGS sources. These data were aggregated to the watershed level outside of this code. 

In [29]:
# Dataset 5: Land use (Falcone et al. 2017a)
rawLU74 = pd.read_csv(inputDataFilepath+'Dataset5_LandUse/LandUse_NWALT_1974.txt', sep=",", dtype={'STAID': str})
rawLU82 = pd.read_csv(inputDataFilepath+'Dataset5_LandUse/LandUse_NWALT_1982.txt', sep=",", dtype={'STAID': str})
rawLU92 = pd.read_csv(inputDataFilepath+'Dataset5_LandUse/LandUse_NWALT_1992.txt', sep=",", dtype={'STAID': str})
rawLU02 = pd.read_csv(inputDataFilepath+'Dataset5_LandUse/LandUse_NWALT_2002.txt', sep=",", dtype={'STAID': str})
rawLU12 = pd.read_csv(inputDataFilepath+'Dataset5_LandUse/LandUse_NWALT_2012.txt', sep=",", dtype={'STAID': str})

# Let's take a look at the dataframes
rawLU74.head()

,STAID,NWALT74_DEV_SUM,NWALT74_SEMIDEV_SUM,NWALT74_AG4344_SUM,NWALT74_AG4346_SUM,NWALT74_11,NWALT74_12,NWALT74_21,NWALT74_22,NWALT74_23,...,NWALT74_31,NWALT74_32,NWALT74_33,NWALT74_41,NWALT74_43,NWALT74_44,NWALT74_45,NWALT74_46,NWALT74_50,NWALT74_60
0,01011000,0.00,0.00,0.01,0.01,3.94,0.79,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.01,0.00,0.00,0.00,94.93,0.33
1,01013500,0.30,0.01,5.68,5.72,6.26,1.86,0.07,0.03,0.01,...,0.0,0.0,0.01,0.01,4.59,1.09,0.01,0.04,84.85,0.99
2,01015800,0.05,0.00,0.16,0.16,2.16,1.85,0.04,0.00,0.00,...,0.0,0.0,0.00,0.00,0.13,0.03,0.00,0.00,94.95,0.82
3,01016500,0.00,0.00,0.13,0.14,0.96,1.58,0.00,0.00,0.00,...,0.0,0.0,0.00,0.00,0.12,0.01,0.00,0.00,97.31,0.00
4,01017000,0.16,0.00,1.91,1.93,1.98,1.72,0.05,0.01,0.00,...,0.0,0.0,0.00,0.01,1.44,0.47,0.01,0.02,93.68,0.51


HINDSIGHT TIP: This is an example of 'wide' data. The date is included in the column name which results in five different textfiles to keep the number of columns within a managable number. It becomes cumberson to process each dataframe the same way. Had this data been 'long' format - a column for Site ID, Year, and land use category - it whave been very simple to join it to clean and join to other data.

In [31]:
# To avoid having repeating code, it's best just to use a function that can dynamically seperate the year from the column names.
def NWALT_dataProcessing(df, year):
    # Set up column names with dynamic year
    colUrb = [f'NWALT{year}_DEV_SUM', f'NWALT{year}_SEMIDEV_SUM']
    colNat = [f'NWALT{year}_11', f'NWALT{year}_12', f'NWALT{year}_50', f'NWALT{year}_60']
    colAg = f'NWALT{year}_AG4346_SUM'

    # Creating new categories
    df[f'NWALT{year}_DEV'] = df[colUrb].sum(axis=1)
    df[f'NWALT{year}_NAT'] = df[colNat].sum(axis=1)

    # Keep only aggregated and agriculture LU columns
    export_df = df.filter([f'STAID', f'NWALT{year}_DEV', f'NWALT{year}_NAT', colAg])

    # Rename the columns
    export_df = export_df.rename(columns={
        "STAID": "SITE",
        f'NWALT{year}_DEV': "UrbLU_pct",
        colAg: "AgLU_pct",
        f'NWALT{year}_NAT': "NatLU_pct"
    })
    return export_df

In [32]:
# Processing the data and then joining the data
LU74 = NWALT_dataProcessing(rawLU74, '74')
LU74['YEAR'] = 1974
LU82 = NWALT_dataProcessing(rawLU82, '82')
LU82['YEAR'] = 1982
LU92 = NWALT_dataProcessing(rawLU92, '92')
LU92['YEAR'] = 1992
LU02 = NWALT_dataProcessing(rawLU02, '02')
LU02['YEAR'] = 2002
LU12 = NWALT_dataProcessing(rawLU12, '12')
LU12['YEAR'] = 2012

LU = pd.concat([LU74, LU82, LU92, LU02, LU12])
LU.head()

,SITE,UrbLU_pct,NatLU_pct,AgLU_pct,YEAR
0,01011000,0.00,99.99,0.01,1974
1,01013500,0.31,93.96,5.72,1974
2,01015800,0.05,99.78,0.16,1974
3,01016500,0.00,99.85,0.14,1974
4,01017000,0.16,97.89,1.93,1974


In [33]:
# Dataset 6: Nitrogen from fertilizer and livestock manure (Falcone et al. 2017a).
rawFERT = pd.read_csv(inputDataFilepath+'Dataset6_N-P_Fertilizer-Manure/N-P_Fertilizer_1987-2012.txt', sep=",", dtype={'STAID': str})
rawMANU = pd.read_csv(inputDataFilepath+'Dataset6_N-P_Fertilizer-Manure/N-P_Manure_1982-2012.txt', sep=",", dtype={'STAID': str})

# Reorganizing fertilizer and manure datasets together in 'long' format.
FERT_l = []
YEARS = range(1987,2012)
for year in YEARS:
    colNames = f'N_FERT_FARM_{year}'
    tempFert = rawFERT.filter(['STAID', colNames])
    tempFert['YEAR'] = year
    tempFert.rename(columns={'STAID':'SITE',
                             colNames: 'FERT_kgkm2'}, inplace=True)
    FERT_l.append(tempFert)
FERT = pd.concat(FERT_l, ignore_index=True)

MANU_l = []
YEARS = range(1982,2012)
for year in YEARS:
    colNames = f'N_MANURE_{year}'
    tempManu = rawMANU.filter(['STAID', colNames])
    tempManu['YEAR'] = year
    tempManu.rename(columns={'STAID':'SITE',
                             colNames: 'MANU_kgkm2'}, inplace=True)
    MANU_l.append(tempManu)

MANU = pd.concat(MANU_l, ignore_index=True)

### Data Cleaning Watershed Attributes
Let's check the data to see if the ranges are reasonable. Also, note that land use and manure data are not recorded yearly, we will have to gap fill this data.

In [51]:
# A quick look at the land use.
grouped_LU = LU.groupby('SITE').agg({
    'YEAR': ['min', 'max'],
    'UrbLU_pct': ['min', 'mean', 'max'],
    'NatLU_pct': ['min', 'mean', 'max'],
    'AgLU_pct': ['min', 'mean', 'max']
})

# Toggle through each land use and metric to make sure they are all over 0% and under 100%. Rounding errors might have occured. 
grouped_LU = grouped_LU.sort_values(by=('UrbLU_pct', 'min'), ascending=True)
grouped_LU.head()

YEAR       UrbLU_pct           NatLU_pct                  AgLU_pct  \
           min   max       min mean  max       min     mean     max      min   
SITE                                                                           
01011000  1974  2012       0.0  0.0  0.0     99.99   99.998  100.00     0.00   
09497700  1974  2012       0.0  0.0  0.0    100.00  100.000  100.00     0.00   
07126415  1974  2012       0.0  0.0  0.0     59.14   59.140   59.14    40.86   
09498503  1974  2012       0.0  0.0  0.0    100.00  100.000  100.00     0.00   
07126390  1974  2012       0.0  0.0  0.0     65.12   65.120   65.12    34.88   

                         
            mean    max  
SITE                     
01011000   0.002   0.01  
09497700   0.000   0.00  
07126415  40.860  40.86  
09498503   0.000   0.00  
07126390  34.880  34.88

In [56]:
# A quick QAQC of the fertilizer and manure data. Checking the max value to make sure they are reasonable magnitudes (under 3000 kg-N km-2 yr-1.)
grouped_FERT = FERT.groupby('SITE').agg({
    'YEAR': ['min', 'max'],
    'FERT_kgkm2': ['min', 'mean', 'max']
})
grouped_FERT = grouped_FERT.sort_values(by=('FERT_kgkm2', 'max'), ascending=False)

grouped_MANU = MANU.groupby('SITE').agg({
    'YEAR': ['min', 'max'],
    'MANU_kgkm2': ['min', 'mean', 'max']
})
grouped_MANU = grouped_MANU.sort_values(by=('MANU_kgkm2', 'max'), ascending=False)
grouped_FERT.head()

YEAR       FERT_kgkm2                    
           min   max        min       mean      max
SITE                                               
06284500  1987  2011     2024.9  14136.780  29939.1
14201300  1987  2011     9184.9  16000.736  24106.8
14192000  1987  2011     6761.2  11766.164  17745.5
06772898  1987  2011     9577.9  12345.540  16434.7
14201340  1987  2011     5501.2   9562.612  14438.4

In [57]:
grouped_MANU.head()

YEAR       MANU_kgkm2                       
           min   max        min          mean      max
SITE                                                  
02108000  1982  2011     3198.8   9165.733333  14796.4
01632082  1982  2011    10304.8  12350.800000  14393.5
01621050  1982  2011     9893.9  11893.083333  13859.3
02106000  1982  2011     1582.3   7816.316667  13725.9
06600300  1982  2011     6168.7   8579.933333  13588.5

### Filling missing data
We don't have land use, manure or fertilizer data for every year of record. It's a good rule of thumb to try not to thin your dataset data, since we usually we want to maximize the available data. Since we want watershed attributes in every given year, we should gap fill our dataset. 

There are a few options. The simplest approach is to use the mean, median, or mode, which are good if you have stationary data and don't care about trends. Alternatively you can also interpolate from year to year for each site. There are much more complex ways to model missing data that are field specific (ex. LOADEST model), and I encourage you to look into those methods.

We might also want to extrapolate beyond available years. Obviously this should be done with caution, but for this first step in data exploration we'll make the assumption that manure and fertilizer inputs don't drastically from year to year. For example, while livestock inventories fluctuate, annual livestock inventories are similar magnitude from year to year because of equipment/housing required for their rearing.  Let's Mean/ Mode/ Median Imputation

In [58]:
# interpolating and extrapolating the missing years for manure and fertilizer. 
YEARS = range(1980, 2020)
SITES = LU['SITE'].unique()
LU_empt = pd.DataFrame({
    'SITE': np.repeat(SITES, len(YEARS)),
    'YEAR': np.tile(YEARS, len(SITES))
})

# Created an new dataframe with the 'NaN' in missing years.
LU_ext = pd.merge(LU_empt, LU, on=['SITE', 'YEAR'], how='left')

# Linearly interpolate missing values 
LU_ext['UrbLU_pct'] = LU_ext.groupby('SITE')['UrbLU_pct'].apply(lambda group: group.interpolate()).reset_index(drop=True)
LU_ext['NatLU_pct'] = LU_ext.groupby('SITE')['NatLU_pct'].apply(lambda group: group.interpolate()).reset_index(drop=True)
LU_ext['AgLU_pct'] = LU_ext.groupby('SITE')['AgLU_pct'].apply(lambda group: group.interpolate()).reset_index(drop=True)

# Linearly extrapolate missing values 
LU_ext['UrbLU_pct'] = LU_ext.groupby('SITE')['UrbLU_pct'].apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
LU_ext['NatLU_pct'] = LU_ext.groupby('SITE')['NatLU_pct'].apply(lambda group: group.ffill().bfill()).reset_index(drop=True)
LU_ext['AgLU_pct'] = LU_ext.groupby('SITE')['AgLU_pct'].apply(lambda group: group.ffill().bfill()).reset_index(drop=True)

In [59]:
# Interpolating and extrapolating the missing years for manure and fertilizer. 
# Manure 
YEARS = range(1980, 2020)
SITES = MANU['SITE'].unique()
MANU_empt = pd.DataFrame({
    'SITE': np.repeat(SITES, len(YEARS)),
    'YEAR': np.tile(YEARS, len(SITES))
})

# Created an new dataframe with the 'NaN' in missing years.
MANU_ext = pd.merge(MANU_empt, MANU, on=['SITE', 'YEAR'], how='left')

# Linearly interpolate missing values 
MANU_ext['MANU_kgkm2'] = MANU_ext.groupby('SITE')['MANU_kgkm2'].apply(lambda group: group.interpolate()).reset_index(drop=True)

# Extrapolate missing values by repeating the nearest values
MANU_ext['MANU_kgkm2'] = MANU_ext.groupby('SITE')['MANU_kgkm2'].apply(lambda group: group.ffill().bfill()).reset_index(drop=True)

# Fertilizer
YEARS = range(1980, 2020)
SITES = FERT['SITE'].unique()
FERT_ext = pd.DataFrame({
    'SITE': np.repeat(SITES, len(YEARS)),
    'YEAR': np.tile(YEARS, len(SITES))
})

# Created an new dataframe with the 'NaN' in missing years.
FERT_ext = pd.merge(FERT_ext, FERT, on=['SITE', 'YEAR'], how='left')

# Extrapolate missing values by repeating the nearest values
FERT_ext['FERT_kgkm2'] = FERT_ext.groupby('SITE')['FERT_kgkm2'].apply(lambda group: group.ffill().bfill()).reset_index(drop=True)

In [151]:
# After interpolating and extrapolating, let's combine the temporal datatogether. 
tAttTable = LU_ext.merge(FERT_ext, on=['SITE','YEAR'], how='left')
tAttTable = tAttTable.merge(MANU_ext, on=['SITE','YEAR'], how='left')

,SITE,YEAR,UrbLU_pct,NatLU_pct,AgLU_pct,FERT_kgkm2,MANU_kgkm2
362675,402913084285400,2015,3.77,3.89,92.34,6042.9,9305.9
362676,402913084285400,2016,3.77,3.89,92.34,6042.9,9305.9
362677,402913084285400,2017,3.77,3.89,92.34,6042.9,9305.9
362678,402913084285400,2018,3.77,3.89,92.34,6042.9,9305.9
362679,402913084285400,2019,3.77,3.89,92.34,6042.9,9305.9


In [75]:
# Let's go ahead and load and compile all the static datasets. 
# Dataset 19: Wastewater treatment plant (point source) density (Falcone et al. 2017b)
rawWWTP = pd.read_csv(inputDataFilepath+'Dataset19_PointSource/GAGE_WWTPDens_20240620.txt', sep=",", dtype={'SITE': str})

# Dataset24: Baseflow index and slope data (Falcone et al. 2017b)
rawBFI = pd.read_csv(inputDataFilepath+'Dataset24_Static/GAGE_BFI_20240620.txt', sep=",", dtype={'SITE': str})
rawSLOPE = pd.read_csv(inputDataFilepath+'Dataset24_Static/GAGE_Slope_pct_20240620.txt', sep=",", dtype={'SITE': str})

# Dataset 99: Aridity Index (Zomer and Trabucco 2022)
rawAI = pd.read_csv(inputDataFilepath+'Dataset99_AridityIndex/GAGE_AI_20240620.txt', sep=",", dtype={'SITE': str})

# Dataset99: DepthGW (Zomer and Trabucco 2022)
rawGWD = pd.read_csv(inputDataFilepath+'Dataset99_DepthGW/GAGE_medGWD_20240620.txt', sep=",", dtype={'SITE': str})

# Dataset99: Fraction of the watershed that is tile-drained
rawTD =  pd.read_csv(inputDataFilepath+'Dataset99_TileDrainage/GAGE_TileDrainge_20240620.txt', sep=",", dtype={'SITE': str})

In [78]:
# Isolating and compiling the remaining static attributes
BFI = rawBFI.filter(['SITE', 'BFI_AVE'])
SLOPE = rawSLOPE.filter(['SITE', 'SLOPE_pct'])
AI = rawAI.filter(['SITE', 'AI'])
GWD = rawGWD.filter(['SITE','medianGWD_m'])
TD = rawTD.filter(['SITE','TD_pct'])

sAttTable = BasinData.merge(BFI, on='SITE', how='left')
sAttTable = sAttTable.merge(SLOPE, on='SITE', how='left')
sAttTable = sAttTable.merge(AI, on='SITE', how='left')
sAttTable = sAttTable.merge(GWD, on='SITE', how='left')
sAttTable = sAttTable.merge(TD, on='SITE', how='left')

In [160]:
# Combine the static properties with the fertilizer, manure, and land use. These are all temporally changing properties. 
# The assumption I am making is that the static variable are constant throughout time.
AttribTable =  tAttTable.merge(sAttTable, on=['SITE'], how='left')

# Now to rearrange the columns in the order I prefer. 
AttribTable = AttribTable[['SITE', 'YEAR', 'NAME',  'LAT', 'LONG', 'STATE', 'AREA_SQKM', 'HUC02',
       'HCDN-2009', 'CLASS', 'AGGECOREGION', 'UrbLU_pct', 'NatLU_pct', 'AgLU_pct', 'FERT_kgkm2',
       'MANU_kgkm2', 'BFI_AVE', 'SLOPE_pct', 'AI',
       'medianGWD_m', 'TD_pct']]
AttribTable.to_csv(inputDataFilepath+'WtshdAttributeTable_20240602.txt',sep=',', index=False)